In [10]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../train/pooled_df.pkl')
df.head()

,SUBJECT_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,...,diagnosisstring,subject_id,stay_id,first_careunit,last_careunit,intime,outtime,los,anchor_age,dod
0,1027140.0,3011529.0,carevue,CCU,CCU,57.0,57.0,2152-07-23 05:34:12,2152-07-24 18:37:32,1.5440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
1,1027482.0,3011175.0,carevue,MICU,MICU,12.0,12.0,2162-03-20 17:22:48,2162-03-22 18:26:10,2.0440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
2,1011746.0,3013234.0,carevue,MICU,MICU,52.0,52.0,2107-07-28 17:22:31,2107-08-01 11:03:44,3.7370,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
3,1010325.0,3022526.0,metavision,MICU,MICU,52.0,52.0,2145-02-06 02:15:08,2145-02-07 18:47:37,1.6892,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
4,1012708.0,3035542.0,metavision,SICU,SICU,33.0,33.0,2175-02-20 07:41:28,2175-02-23 10:21:01,3.1108,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT


In [2]:
print(df.columns)

Index(['SUBJECT_ID', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG',
       'age', 'readmission', 'mortality', 'los_3day', 'los_7day', 'ICD9_CODE',
       '12h_obs', '24h_obs', 'ID', 'code_name', 'code_offset', 'value', 'uom',
       'code_order', 'seq_len'],
      dtype='object')


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch


# Load the pre-trained model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed(input_text):
    tokens = tokenizer(input_text, return_tensors="pt")
    embeddings = model(**tokens).last_hidden_state
    pooled_embeddings = embeddings.mean(dim=1)
    return pooled_embeddings

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading: 100%|██████████| 436M/436M [00:31<00:00, 13.9MB/s] 
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def embed_data(df):
    # Create empty numpy array with len(df) rows and 256 columns
    df_embed = []
    for k in range(len(df)):
        df_embed.append([])
        for n in range(len(df['code_name'][k])):
            text_features = str(df['code_name'][k][n]) + ' ' + str(df['uom'][k][n])
            value_features = [d for d in str(df['value'][k][n])]
            # Embed features separately then concatenate the tensors
            text_features = [embed(feature) for feature in text_features]
            value_features = [embed(feature) for feature in value_features]
            embed_input = torch.cat(text_features + value_features, dim=1)
            # Save to df
            df_embed[k].append(embed_input)
    return pd.DataFrame(df_embed)

# Test with first 10 rows
df_test = df.head(2)
df_test = embed_data(df_test)
df_test.head()

# Save to pickle
#df_test.to_pickle('df_test.pkl')

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,"[[tensor(0.4792, grad_fn=<UnbindBackward0>), t...","[[tensor(0.1914, grad_fn=<UnbindBackward0>), t...","[[tensor(0.1914, grad_fn=<UnbindBackward0>), t...","[[tensor(0.4965, grad_fn=<UnbindBackward0>), t...","[[tensor(0.4965, grad_fn=<UnbindBackward0>), t...","[[tensor(0.4965, grad_fn=<UnbindBackward0>), t...","[[tensor(0.4965, grad_fn=<UnbindBackward0>), t...","[[tensor(0.4965, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2712, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2643, grad_fn=<UnbindBackward0>), t...",...,"[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t..."
1,"[[tensor(0.2717, grad_fn=<UnbindBackward0>), t...","[[tensor(0.6810, grad_fn=<UnbindBackward0>), t...","[[tensor(1.1748, grad_fn=<UnbindBackward0>), t...","[[tensor(0.5084, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2430, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2712, grad_fn=<UnbindBackward0>), t...","[[tensor(0.6810, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2712, grad_fn=<UnbindBackward0>), t...","[[tensor(1.1748, grad_fn=<UnbindBackward0>), t...","[[tensor(0.5084, grad_fn=<UnbindBackward0>), t...",...,"[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0328, grad_fn=<UnbindBackward0>), t..."
